In [219]:
import random
import math
import arcade

In [220]:
def get_pos_at_range(r):
    
    list_pos = []
    
    for i in range(r + 1):
        if i != 0:
            list_pos.append((0, i))
            list_pos.append((0, -i))
    
        for j in range(r - i):
            list_pos.append((1 + j, i))
            list_pos.append((-1 - j, i))
            
            if i != 0:
                list_pos.append((1 + j, -i))
                list_pos.append((-1 - j, -i))
    
    return list_pos

In [221]:
"""
#  #  #  #  #  #  1  #  #  #  #  #  #
#  #  #  #  #  2  3  4  #  #  #  #  #
#  #  #  #  5  6  7  8  9  #  #  #  #
#  #  #  10 11 12 13 14 15 16 #  #  #
#  #  17 18 19 20 21 22 23 24 25 #  #
#  26 27 28 29 30 31 32 33 34 35 36 #
37 38 39 40 41 42 -- 44 45 46 47 48 49
#  50 51 52 53 54 55 56 57 58 59 60 #
#  #  61 62 63 64 65 66 67 68 69 #  #
#  #  #  70 71 72 73 74 75 76 #  #  #
#  #  #  #  77 78 79 80 81 #  #  #  #
#  #  #  #  #  82 83 84 #  #  #  #  #
#  #  #  #  #  #  85 #  #  #  #  #  #
"""

END_TURN = -1
ACTIONS = get_pos_at_range(6)

ACTIONS.append(END_TURN)
print(len(ACTIONS))

85


In [222]:
REWARD_MOB = 16
REWARD_END_TURN = -2
REWARD_MOVE = -4
REWARD_IMPOSSIBLE = -128
REWARD_END_FIGHT = 64

EMPTY_CELL = 0
WALL = 1
USABLE_CELL = 2
PLAYER = 3
MONSTER = 4
INVO = 5
BOSS = 6
UNDEFINED = -1

In [223]:
l = 20
h = 14

base_x = 235
base_y = 5

step_x = 130
step_y = 65

box_size = 30

dict_swap_dofus_to_pos = {}
dict_swap_pos_to_dofus  = {}

dict_swap_dofus_to_array_index  = {}
dict_swap_pos_to_array_index  = {}

dict_swap_array_index_to_dofus  = []
dict_swap_array_index_to_pos  = []
index = 0

for j in range(h):
    
    x_start = 13 - j
    
    for i in range(l):
        
        dict_swap_dofus_to_pos[(j, i, False)] = (x_start + i, i + j)
        dict_swap_dofus_to_pos[(j, i, True)] = (x_start + i, i + j + 1)
        
        dict_swap_pos_to_dofus[(x_start + i, i + j)] = (j, i, False)
        dict_swap_pos_to_dofus[(x_start + i, i + j + 1)] = (j, i, True)
        
        dict_swap_dofus_to_array_index[(j, i, False)] = index
        dict_swap_pos_to_array_index[(x_start + i, i + j)] = index
        
        dict_swap_array_index_to_dofus.append((j, i, False))
        dict_swap_array_index_to_pos.append((x_start + i, i + j))
        index += 1

        dict_swap_dofus_to_array_index[(j, i, True)] = index
        dict_swap_pos_to_array_index[(x_start + i, i + j + 1)] = index
        dict_swap_array_index_to_dofus.append((j, i, True))
        dict_swap_array_index_to_pos.append((x_start + i, i + j + 1))
        
        index += 1
        

In [224]:
def get_pos_at_range_possible(dict_map, start, r):
    
    list_pos = get_pos_at_range(r)
    full_list_pos = []
    for i in range(len(list_pos)):
        
        list_pos[i] = list_pos[i][0] + start[0], list_pos[i][1] + start[1]
        
        if list_pos[i] not in dict_swap_pos_to_array_index.keys():
            continue
    
        if dict_map[dict_swap_pos_to_array_index[list_pos[i]]] == USABLE_CELL:
            full_list_pos.append(list_pos[i])

    return full_list_pos

In [225]:
def get_map(path):
    
    dict_map = {}
    array_map = []
    list_state_map = []
    
    with open(path, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            array_map.append(list(map(int, line.split(','))))
    
    for pos in dict_swap_pos_to_dofus.keys():
        dict_map[pos] = array_map[pos[0]][pos[1]]
        
    for i in range(h):
        for j in range(l):
            list_state_map.append(dict_map[dict_swap_dofus_to_pos[(i, j, False)]])
            list_state_map.append(dict_map[dict_swap_dofus_to_pos[(i, j, True)]])
    
    return list_state_map

In [226]:
def get_random_start_map(dict_map):
    
    list_keys_player = []
    list_keys_mob = []
    
    for key in range(len(dict_map)):
        if dict_map[key] == 3:
            list_keys_player.append(key)
        elif dict_map[key] == 4:
            list_keys_mob.append(key)
            
    i = random.randint(0, len(list_keys_player) - 1)
    list_keys_player.remove(list_keys_player[i])
    
    for _ in range(4):
        i = random.randint(0, len(list_keys_mob) - 1)
        list_keys_mob.remove(list_keys_mob[i])
        
    for pos in list_keys_player:
        dict_map[pos] = 2
        
    for pos in list_keys_mob:
        dict_map[pos] = 2
        
    return dict_map

In [227]:
def get_player_pos(dict_map):
    
    for pos in range(len(dict_map)):
        if dict_map[pos] == 3:
            return dict_swap_array_index_to_pos[pos]
        
    return -1

In [228]:
def mob_turn(dict_map, boss_map=False):
    
    list_mob = []
    have_invo = False
    
    for pos in range(len(dict_map)):
        if dict_map[pos] == MONSTER:
            list_mob.append(pos)
        elif dict_map[pos] == INVO:
            have_invo = True
    
    for mob in list_mob:
        
        possible_pos = get_pos_at_range_possible(dict_map, dict_swap_array_index_to_pos[mob], 3)
        
        if len(possible_pos) == 0:
            continue

        dict_map[mob] = USABLE_CELL
        new_pos = dict_swap_pos_to_array_index[possible_pos[random.randint(0, len(possible_pos) - 1)]]
        dict_map[new_pos] = MONSTER

    return dict_map

In [229]:
def map_contains_mob(dict_map):

    for pos in range(len(dict_map)):
        if dict_map[pos] == MONSTER or dict_map[pos] == INVO or dict_map[pos] == BOSS:
            return True

    return False

In [230]:
def ldv(dict_map, from_pos, to_pos):
    
    diff = [to_pos[0] - from_pos[0], to_pos[1] - from_pos[1]]
    tmp_from = [from_pos[0], from_pos[1]]
    
    if abs(diff[0]) > abs(diff[1]):
        index_factor = 0
        index_float = 1
    else:
        index_factor = 1
        index_float = 0

    factor = diff[0] / abs(diff[index_factor]), diff[1] / abs(diff[index_factor])
    
    while tmp_from[0] != to_pos[0] or tmp_from[1] != to_pos[1]:
        

        if not float(tmp_from[index_float]).is_integer():
            
            up = [0, 0]
            down = [0, 0]
            
            up[index_factor] = tmp_from[index_factor]
            down[index_factor] = tmp_from[index_factor]
            
            up[index_float] = math.ceil(tmp_from[index_float])
            down[index_float] = math.floor(tmp_from[index_float])
            
            if tuple(up) in dict_swap_pos_to_array_index.keys():
                up_index = dict_swap_pos_to_array_index[tuple(up)]            
                if dict_map[up_index] == WALL or dict_map[up_index] == MONSTER or dict_map[up_index] == BOSS or dict_map[up_index] == INVO:
                    return False
            
            if tuple(down) in dict_swap_pos_to_array_index.keys():
                down_index = dict_swap_pos_to_array_index[tuple(down)]
                if dict_map[down_index] == WALL or dict_map[down_index] == MONSTER or dict_map[down_index] == BOSS or dict_map[down_index] == INVO:
                    return False
        else:
            
            tmp_index = dict_swap_pos_to_array_index[tuple(tmp_from)]
            
            if dict_map[tmp_index] == WALL or dict_map[tmp_index] == MONSTER or dict_map[tmp_index] == BOSS or dict_map[tmp_index] == INVO:
                return False
            
        tmp_from[0] += factor[0]
        tmp_from[1] += factor[1]
    
    return True

In [231]:
def can_move(dict_map, from_pos, to_pos):
    
    diff = [to_pos[0] - from_pos[0], to_pos[1] - from_pos[1]]
    tmp_from = [from_pos[0], from_pos[1]]
    
    if abs(diff[0]) > abs(diff[1]):
        index_factor = 0
        index_float = 1
    else:
        index_factor = 1
        index_float = 0

    factor = diff[0] / abs(diff[index_factor]), diff[1] / abs(diff[index_factor])
    
    while tmp_from[0] != to_pos[0] and tmp_from[1] != to_pos[1]:
        if not float(tmp_from[index_float]).is_integer():
            
            up = [0, 0]
            down = [0, 0]
            
            up[index_factor] = tmp_from[index_factor]
            down[index_factor] = tmp_from[index_factor]
            
            up[index_float] = math.ceil(tmp_from[index_float])
            down[index_float] = math.floor(tmp_from[index_float])
            
            if tuple(up) in dict_swap_pos_to_array_index.keys():
                up_index = dict_swap_pos_to_array_index[tuple(up)]            
                if dict_map[up_index] == WALL or dict_map[up_index] == MONSTER or dict_map[up_index] == BOSS or dict_map[up_index] == INVO:
                    return False
            
            if tuple(down) in dict_swap_pos_to_array_index.keys():
                down_index = dict_swap_pos_to_array_index[tuple(down)]
                if dict_map[down_index] == WALL or dict_map[down_index] == MONSTER or dict_map[down_index] == BOSS or dict_map[down_index] == INVO:
                    return False
            
        else:
            
            tmp_index = dict_swap_pos_to_array_index[tuple(tmp_from)]
            
            if dict_map[tmp_index] != USABLE_CELL:
                return False

        tmp_from[0] += factor[0]
        tmp_from[1] += factor[1]
    
    return True

In [232]:
def count_pm_to_pos(dict_map, player_pos, expected_pos, max_pm):
    
    diff = expected_pos[0] - player_pos[0], expected_pos[1] - player_pos[1]
    dist = abs(diff[0]) + abs(diff[1])
    
    if dist < max_pm and can_move(dict_map, player_pos, expected_pos):
        return dist

    return -1

In [233]:
class Environment:
    def __init__(self):
        
        self.map_index = random.randint(0, 4)
        self.map = get_random_start_map(get_map(f"./data/map_array/map_{self.map_index + 1}.txt"))
        
        self.max_pa = 6
        self.max_pm = 3
        
        self.pa = self.max_pa
        self.pm = self.max_pm
        
    def do(self, action):
        
        clicked = ACTIONS[action]
        player_pos = get_player_pos(self.map)
        
        if player_pos == -1:
            exit()
        
        if clicked == END_TURN:
            self.pa = self.max_pa
            self.pm = self.max_pm
            self.map = mob_turn(self.map, boss_map=False)
            return REWARD_END_TURN
        
        clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]
        
        if clicked_pos not in dict_swap_pos_to_array_index.keys():
            return REWARD_IMPOSSIBLE
        
        clicked_index = dict_swap_pos_to_array_index[clicked_pos]

        if self.map[clicked_index] == EMPTY_CELL or self.map[clicked_index] == WALL or self.map[clicked_index] == UNDEFINED:
            return REWARD_IMPOSSIBLE
        
        elif self.map[clicked_index] == MONSTER or self.map[clicked_index] == INVO or self.map[clicked_index] == BOSS:
            
            if self.pa < 3:
                return REWARD_IMPOSSIBLE
            
            if not ldv(self.map, player_pos, clicked_pos):
                return REWARD_IMPOSSIBLE
            
            self.map[clicked_index] = USABLE_CELL
            self.pa -= 3
            
            if not map_contains_mob(self.map):
                return REWARD_END_FIGHT
            return REWARD_MOB
        
        elif self.map[clicked_index] == USABLE_CELL:
            used = count_pm_to_pos(self.map, player_pos, clicked_pos, self.pm)
            
            if used == -1:
                return REWARD_IMPOSSIBLE
            
            self.map[clicked_index] = PLAYER
            self.map[dict_swap_pos_to_array_index[player_pos]] = USABLE_CELL
            self.pm -= used
            return REWARD_MOVE

    @property
    def state(self):
        return tuple(self.map), self.pa, self.pm

In [234]:
class Agent:
    def __init__(self):
        self.env = Environment()
        self.score = 0
        self.qtable = {}

    def reset(self):
        self.env = Environment()
        self.score = 0
        
    def best_action(self):
        
        if self.env.state not in self.qtable:
            self.qtable[self.env.state] = {}
            for action in range(len(ACTIONS)):
                self.qtable[self.env.state][action] = 0.0

        return max(self.qtable[self.env.state], key=self.qtable[self.env.state].get)
    
    def do(self):
        state = self.env.state
        action = self.best_action()
        
        reward = self.env.do(action)
        
        self.qtable[state][action] += reward

        self.score += reward
        return reward

In [235]:
class DofusStatusWindow(arcade.Window):
    
    def __init__(self):
        super().__init__(2256 // 2, 1504 // 2, "DofusStatus")
        arcade.set_background_color(arcade.color.BLACK)
        self.agent = Agent()
        
    def draw_box(self, x, y, color):
        arcade.draw_rectangle_filled(x, 1504 // 2 - y, box_size // 2, box_size // 2, color)
        
    def on_draw(self):
        arcade.start_render()
        for pos in range(len(self.agent.env.map)):
            if self.agent.env.map[pos] != 0:

                dofus_pos = dict_swap_array_index_to_dofus[pos]

                x = base_x + step_x * dofus_pos[0]
                y = base_y + step_y * dofus_pos[1]
                
                if dofus_pos[2]:
                    x += step_x // 2
                    y += step_y // 2
                
                x = x // 2
                y = y // 2
                
                if self.agent.env.map[pos] == 1:
                    self.draw_box(x, y, arcade.color.GRAY)
                elif self.agent.env.map[pos] == 2:
                    self.draw_box(x, y, arcade.color.GREEN)
                elif self.agent.env.map[pos] == 3:
                    self.draw_box(x, y, arcade.color.BLUE)
                elif self.agent.env.map[pos] == 4:
                    self.draw_box(x, y, arcade.color.RED)
                elif self.agent.env.map[pos] == 5:
                    self.draw_box(x, y, arcade.color.YELLOW)
                elif self.agent.env.map[pos] == 6:
                    self.draw_box(x, y, arcade.color.ORANGE)
                else:
                    self.draw_box(x, y, arcade.color.AQUA)

    def on_update(self, delta_time):
        r = self.agent.do()
        if r == REWARD_END_FIGHT:
            self.agent.reset()


In [236]:
window = DofusStatusWindow()
arcade.run()

In [ ]:
agent = Agent()
agent.score = - 1
cnt = 0

while agent.score < 0:

    agent.reset()
    
    while agent.do() != REWARD_END_FIGHT:
        continue

    cnt += 1
    print(f"Score {cnt} : {agent.score}")
    

Score 1 : -640466
Score 2 : -691584
Score 3 : -731298
Score 4 : -776456
Score 5 : -465094
Score 6 : -1028692
Score 7 : -2228100
Score 8 : -1692824
Score 9 : -960114
Score 10 : -695406
Score 11 : -1059762
Score 12 : -1160710
Score 13 : -819052
Score 14 : -437146
Score 15 : -1490658
Score 16 : -894462
Score 17 : -787066
Score 18 : -1137510
Score 19 : -526190
Score 20 : -1232884
Score 21 : -385654
Score 22 : -311452
Score 23 : -380136
Score 24 : -792560


In [ ]:
with open('./pod/qtable.json', 'w') as f:
    f.write(str(agent.qtable))

In [ ]:
cnt = 1

while 1:
    
    agent.reset()
    
    while agent.do() != REWARD_END_FIGHT:
        continue
        
    print(f"Score {cnt} : {agent.score}")
    if cnt % 5000 == 0:
        with open(f"./pod/qtable-{cnt // 5000}.json", 'w') as f:
            f.write(str(agent.qtable))

    cnt += 1